In [1]:
# Insérez ici le numéro de votre équipe
VOTRE_NUMERO_EQUIPE = 4

In [2]:
# Import des paquets requis et configuration du ROS Master
import sys
sys.path.append('/opt/ros/noetic/lib/python3/dist-packages/')
sys.path.append('/usr/lib/python3/dist-packages')
from os import environ
environ['ROS_MASTER_URI'] = "http://localhost:1137{}/".format(VOTRE_NUMERO_EQUIPE)
import rospy
from jackal_msgs.msg import Drive

### Configuration du script

Ajoutez ici les valeurs géométriques du robot

In [3]:
# Compléter les variables suivantes avec les dimensions du robot en [m]
interwheel_distance = 0.46
left_wheel_radius = 0.045
right_wheel_radius = 0.045

In [4]:
# Créer et démarrer un nouveau noeud
rospy.init_node('dingo_controller', anonymous=True)

In [5]:
# ROS subscribers et publishers
cmd_drive_pub = rospy.Publisher('/mobile_manip/dingo_velocity_controller/cmd_drive', Drive, queue_size=1)

### Explication du calcul

#### A partir des deux equations suivantes

vit_lin = (v_d + v_g) / 2

vit_ang = (v_d - v_g) / inter_wheel



#### On peut isoler les valeurs des vitesses de roues de droite et gauche

v_d = vit_lin + (inter_wheel * vit_ang / 2)

v_g = vit_lin - (inter_wheel * vit_ang / 2)


#### On traduit les vitesse linéaires des rous en vitesses angulaires pour la commande

v_d_ang = (vit_lin + (inter_wheel * vit_ang) / 2) / right_wheel_radius

v_g_ang = (vit_lin - (inter_wheel * vit_ang) / 2) / left_wheel_radius

### Cinématique inverse

In [6]:
# Calcule les commandes à envoyer à chaque moteur-roue selon le mouvement désiré
def move_robot(linear_vel, angular_vel):

    # Tâche 2
    # Vous devez compléter ici avec la cinématique inverse en termes de vitesses
    # c'est-à-dire obtenir les vitesses des roues à partir des vitesses angulaire (angular_vel) et linéaire (linear_vel) désirées
        
    vel_left  = (linear_vel-(interwheel_distance*angular_vel/2))/left_wheel_radius
    vel_right = (linear_vel+(interwheel_distance*angular_vel/2))/right_wheel_radius
    

    # Envoi des commandes au roues par topic ROS - ne pas modifier
    cmd_drive_msg = Drive()

    cmd_drive_msg.drivers[0] = vel_left
    cmd_drive_msg.drivers[1] = vel_right

    cmd_drive_pub.publish(cmd_drive_msg)

### Test et validation

In [7]:
# Création d'une interface minimaliste pour valider la cinématique inverse - ne pas modifier
import ipywidgets as widgets
float_linear = widgets.FloatSlider(description='Linear velocity:', min=-10.0, max=10.0, step=0.01)
float_angular = widgets.FloatSlider(description='Angular velocity:', min=-5.0, max=5.0, step=0.01)
output_linear = widgets.Output()
output_angular = widgets.Output()

display(float_linear, output_linear)
display(float_angular, output_angular)

def on_value_change(change):
    with output_linear:
        linear = float_linear.value
    with output_angular:
        angular = float_angular.value
    move_robot(linear, angular)

float_linear.observe(on_value_change, names='value')
float_angular.observe(on_value_change, names='value')

#Ci-dessous des curseurs permettant d'envoyer immédiatement des valeurs de vitesse linéaire et angulaire au Jackal.

FloatSlider(value=0.0, description='Linear velocity:', max=10.0, min=-10.0, step=0.01)

Output()

FloatSlider(value=0.0, description='Angular velocity:', max=5.0, min=-5.0, step=0.01)

Output()